## Week 11 Practice

### Problem 1: Implementing an objective function in NumPy and PyTorch
Implement two Python functions that implements the mathematical functions $f(x) = 3x^\intercal x - x_1 - 4$. One function should work for $x$ a `numpy.ndarray` and the other should work for $x$ a `torch.FloatTensor`.

In [10]:
import torch
import numpy as np


def f_numpy(x):
    return 3 * np.dot(x, x) - x[0] - 4


def f_torch(x):
    return 3 * torch.dot(x, x) - x[0] - 4


In [11]:
assert abs(f_numpy(np.zeros(4)) + 4) < 1e-6
assert abs(f_numpy(np.ones(2)) - 1) < 1e-6

assert abs(f_torch(torch.zeros(4)) + 4) < 1e-6
assert abs(f_torch(torch.ones(2)) - 1) < 1e-6

x0 = (1, -2, 3, 4, 5)
assert abs(f_numpy(np.array(x0)) - f_torch(torch.Tensor(x0))) < 1e-6

### Problem 2: Computing gradients manually and automatically
Implement functions to compute the gradient of $f$ at $x$ using numpy and PyTorch. Use autodiff for the latter.

In [23]:
def f_grad_numpy(x):
    gradient = 6 * x
    gradient[0] -= 1
    return gradient


def f_grad_torch(x):
    f_x = 3 * torch.dot(x, x) - x[0] - 4
    gradient = torch.autograd.grad(f_x, x)[0]
    return gradient

In [24]:
def finite_diff(x, v):
    return (f_numpy(x + v) - f_numpy(x - v)) / (2 * np.linalg.norm(v))


x = np.ones(2)
v0 = 1e-4 * np.array([1, 0])
assert abs(finite_diff(x, v0) - f_grad_numpy(x)[0]) < 1e-2
v1 = 1e-4 * np.array([0, 1])
assert abs(finite_diff(x, v1) - f_grad_numpy(x)[1]) < 1e-2

np.random.seed(42)
for i in range(10):
    x2 = np.random.randn(5)
    v2 = np.random.randn(5)
    v2 = v2 / np.linalg.norm(v2)
    observed = finite_diff(x2, 1e-4 * v2)
    derived_vec = f_grad_numpy(x2)
    derived = derived_vec.dot(v2)
    assert abs(observed - derived) < 1e-2

xt = torch.tensor(x, requires_grad=True)
diff = f_grad_torch(xt).numpy() - f_grad_numpy(x)
assert np.linalg.norm(diff) < 1e-6